## Dataset Setup

In [1]:
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
import datasets

liar = datasets.load_dataset("liar")
liar

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 10269
    })
    test: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1283
    })
    validation: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1284
    })
})

In [4]:
train = liar["train"]
test = liar["test"]
val = liar["validation"]

## Model Loading

In [6]:
falcon = "tiiuae/falcon-7b-instruct"
llama = "meta-llama/Llama-2-7b-chat-hf"
mistral = "mistralai/Mistral-7B-Instruct-v0.2"
orca = "microsoft/Orca-2-7b"

In [7]:
# change this depending on experiment
model_name = llama

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=config, device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Experiment

### Experiment Utils

In [47]:
from typing import Dict


def to_prompt(entry: Dict[str, str]) -> str:
    speaker = entry["speaker"].replace("-", " ").title()
    statement = entry["statement"].lstrip("Says ")

    prompt = f"""Please select the option that most closely describes the following claim by {speaker}:\n{statement}\n\nA) True\nB) Mostly True\nC) Half True\nD) Barely True\nE) False\nF) Pants on Fire (absurd lie)\n\nChoice: ("""
    return prompt

In [48]:
from transformers import PreTrainedTokenizer

LABEL_MAP = {
    0: "E",  # 0 : False
    1: "C",  # 1 : Half True
    2: "B",  # 2 : Mostly True
    3: "A",  # 3 : True
    4: "D",  # 4 : Barely True
    5: "F",  # 5 : Pants on Fire
}


def was_correct(
    generation: torch.Tensor, tokenizer: PreTrainedTokenizer, entry: Dict[str, int]
) -> bool:
    response = tokenizer.decode(generation[0, -1])
    return LABEL_MAP[entry["label"]] in response

In [49]:
def workflow(entry: dict, model, verbose: bool = False):
    prompt = to_prompt(entry)
    generation = model.generate(
        tokenizer(prompt, return_tensors="pt").input_ids.cuda(), max_new_tokens=1, do_sample=False
    )

    # log the prompt and response if verbose
    if verbose:
        print(tokenizer.batch_decode(generation)[0])

    correct = was_correct(generation, tokenizer, entry)

    if verbose:
        print(
            "The model was",
            "correct" if correct else "incorrect",
            " - responded",
            tokenizer.decode(generation[0, -1]),
            "and answer should have been",
            LABEL_MAP[entry["label"]],
        )

### Verify it works

In [61]:
import random

workflow(train[random.randint(0, len(train)-1)], model, verbose=True)

<s> Please select the option that most closely describes the following claim by Mitt Romney:
President Ronald Reagan sent troops into conflict only in one circumstance, which was in Grenada We were in a peacekeeping setting in Lebanon.

A) True
B) Mostly True
C) Half True
D) Barely True
E) False
F) Pants on Fire (absurd lie)

Choice: (D
The model was  correct  - responded D and answer should have been D


### Run Experiment